# American Options

## Black-scholes

In [2]:
import time
import pandas as pd
import numpy as np
def measure_time(func, *args, **kwargs):
    start_time = time.time()  
    result = func(*args, **kwargs) 
    end_time = time.time()  
    execution_time = end_time - start_time  # Calculate the time difference
    return result, execution_time
def crr(S0, K, T, r, sigma, n):

    dt = T / n  
    u = np.exp(sigma * np.sqrt(dt))  
    d = np.exp(-sigma * np.sqrt(dt)) 
    p = (np.exp(r * dt) - d) / (u - d)  
    

    asset_prices = np.zeros((n + 1, n + 1))
    for j in range(n + 1):
        for i in range(j + 1):
            asset_prices[i, j] = S0 * (u ** (j - i)) * (d ** i)
    

    option_prices = np.zeros((n + 1, n + 1))
    

    for i in range(n + 1):
        option_prices[i, n] = max(0, K-asset_prices[i, n] )
    
  
    for j in range(n - 1, -1, -1):
        for i in range(j + 1):
            option_prices[i, j] = np.exp(-r * dt) * (p * option_prices[i, j + 1] + (1 - p) * option_prices[i + 1, j + 1])
            
            option_prices[i, j] = max(option_prices[i, j], K-asset_prices[i, j] )

    return option_prices[0, 0]

In [3]:
import numpy as np
from scipy.linalg import lu
from scipy.interpolate import interp1d, interp2d
from scipy import sparse
from scipy.sparse.linalg import splu
from scipy.sparse.linalg import spsolve
def pde(S0, K, r,q, Time, sigma, M, N,theta,Smax):

    matval = np.zeros((M + 1, N + 1))
    S, dS = np.linspace(0, Smax, M + 1, retstep=True)  # space discretization
    T, dt = np.linspace(0, Time, N + 1, retstep=True)  # time discretization
    dSS=dS*dS
    j=T/dt
    # set up boundary conditions
    U_0 = np.maximum( K-S, 0)
    matval[:, N] = U_0
    matval[0, :] = K * np.exp(-r * (Time-dt * (j)))
    matval[M, :] = 0
    a=np.zeros(M+1)
    b=np.zeros(M+1)
    c = np.zeros(M + 1)
    for i in range(M+1):
        a[i] = 0.5 * (sigma** 2) * (S[i] **2);
        b[i] = (r - q) * S[i];
        c[i] = -r;
    alpha = (dt / dSS) * a[1:M] - dt  / (2 * dS) * b[1:M];
    beta = -2 * (dt / dSS) * a[1:M] + dt  * c[1:M];
    gamma =  (dt / dSS) * a[1:M] + dt  / (2 * dS) * b[1:M];
    D = sparse.diags([alpha[1:], beta, gamma[:M-1]], [-1, 0, 1], shape=(M-1, M-1), format='csr')
    I=sparse.eye(M-1, format='csr')
    A=I-theta*D
    B=I+(1-theta)*D
    aux1 = np.zeros(M-1)
    aux2 = np.zeros(M - 1)
    Array=np.zeros(N+1)
    lamda=U_0[1:M,]*0
    for j in range(N - 1, -1, -1):
        aux1[M-2]=-theta*gamma[-1]* matval[M,j+1]
        aux1[0] = -theta*alpha[0] * matval[0, j+1]
        aux2[M - 2] = (1-theta) * gamma[-1] * matval[M, j ]
        aux2[0] = (1-theta) * alpha[0] * matval[0, j ]
        matval[1:M, j] = spsolve(A, B@matval[1:M, j + 1]-aux1+aux2+dt*lamda)
        lamda_=lamda
        lamda = np.maximum(0, lamda_ + (U_0[1:M,] - matval[1:M, j]) / dt)
        matval[1:M, j] = np.maximum( matval[1:M, j] - dt* lamda_, U_0[1:M,]) 
    f=interp1d(S, matval[:,0], kind='linear', fill_value='extrapolate')    
    return f(S0) 

In [4]:
S0=100
K=100
T=0.5
r=0.02
sigma=0.15
n=1000


In [5]:
def longstaff(S0,K,T,r,sigma):
    N=300
    M=200000
    dt = T / (N - 1)  # time step
    df = np.exp(-r * dt)
    def GBM_Process(maturity, nSims, drift, spot_price, sigma, dt):
        spotsims = np.zeros((  maturity,nSims))
        spotsims[0, :] = spot_price
        for t in range(1, maturity):
            z = np.random.standard_normal(nSims)
            spotsims[t, :] = spotsims[t-1, :] * np.exp((drift - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)
        return spotsims
    S=GBM_Process(N, M, r, S0, sigma, dt)
    def lsm(S,K,r,t):
        
        N=S.shape[0]
        dt = T / (N - 1)

        CF = np.zeros_like(S)
        CF[-1, :] = np.maximum(K - S[-1, :], 0)


        for ii in range(N - 2, 0, -1):
            idx = np.where(S[ii, :] < K)[0]
    
    
            X = S[ii, idx]
            NX=len(X)
            X1 = X / S0
            Y = CF[ii + 1, idx] * np.exp(-r * dt)
    
            # Linear regression step
            R = np.vstack([np.ones_like(X1), (1 - X1), 1/2 * (2 - 4 * X1 - X1**2)])
            a = np.linalg.lstsq(R.T, Y, rcond=None)[0]
    
            C = R.T @ a
    
            jdx = np.where(np.maximum(K - X, 0) > C)[0]
            nidx = np.setdiff1d(np.arange(M), idx[jdx])
    
            CF[ii, idx[jdx]] = np.maximum(K - X[jdx], 0)
            CF[ii, nidx] = np.exp(-r * dt) * CF[ii + 1, nidx]

        return np.mean(CF[1, :]) * np.exp(-r * dt)

    return (lsm(S,K,r,T))

In [6]:

args_f = (S0, K, T, r, sigma, n)  
args_g = (S0, K, r,0, T, sigma, 100000, 300,0.5,1000)  
args_h = (S0,K,T,r,sigma) 


results = {
    'Function': ['crr', 'pde', 'longstaff'],
    'Execution Time (s)': []
}


value_f, exec_time_f = measure_time(crr, *args_f)
value_g, exec_time_g = measure_time(pde, *args_g)
value_h, exec_time_h = measure_time(longstaff, *args_h)


results['Value'].append(value_f)
results['Execution Time (s)'].append(exec_time_f)

results['Value'].append(value_g)
results['Execution Time (s)'].append(exec_time_g)

results['Value'].append(value_h)
results['Execution Time (s)'].append(exec_time_h)


df = pd.DataFrame(results)


print(df)

    Function               Value  Execution Time (s)
0        crr            3.809832            2.584014
1        pde  3.8104172097043696            2.882039
2  longstaff            3.799278           17.055240


In [8]:
S0=100
K=110
T=0.5
r=0.02
sigma=0.15
n=1000
args_f = (S0, K, T, r, sigma, n)  
args_g = (S0, K, r,0, T, sigma, 100000, 300,0.5,1000)  
args_h = (S0,K,T,r,sigma) 


results = {
    'Function': ['crr', 'pde', 'longstaff'],
    'Value': [],
    'Execution Time (s)': []
}


value_f, exec_time_f = measure_time(crr, *args_f)
value_g, exec_time_g = measure_time(pde, *args_g)
value_h, exec_time_h = measure_time(longstaff, *args_h)


results['Value'].append(value_f)
results['Execution Time (s)'].append(exec_time_f)

results['Value'].append(value_g)
results['Execution Time (s)'].append(exec_time_g)

results['Value'].append(value_h)
results['Execution Time (s)'].append(exec_time_h)


df = pd.DataFrame(results)


print(df)

    Function               Value  Execution Time (s)
0        crr           10.570537            2.533360
1        pde  10.570494696983094           33.327374
2  longstaff           10.515856           17.108058
